## 데이터 전처리 부분은 책 코드 사용

In [ ]:
from google.colab import drive
drive.mount('./mount')

Mounted at ./mount


In [ ]:
import torch
if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda:", torch.cuda.get_device_name(0))
  pass

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cpu')

In [ ]:
import os
os.path.curdir

'.'

In [ ]:
from .mount.My Drive.Colab Notebooks.utils.write import training_data, test_data, collapse_documents, expand_answers, _read_data, glove
training_data_gen = training_data()
# training_data_gen = [next(training_data_gen)]
test_data_gen = test_data()


SyntaxError: ignored

In [ ]:
t = next(training_data_gen)

In [ ]:
idx = 0

print('document_tokens\n', t['document_tokens'][idx])
print('\n')
print('question_input_tokens\n', t['question_input_tokens'][idx])
print('\n')
print('answer_masks\n', t['answer_masks'][idx])
print('\n')
print('answer_labels\n', t['answer_labels'][idx])
print('\n')
print('question_output_tokens\n', t['question_output_tokens'][idx])

In [ ]:
# GloVe

VOCAB_SIZE = glove.shape[0]
EMBEDDING_DIMENS = glove.shape[1]

print('GLOVE')
print('VOCAB_SIZE: ', VOCAB_SIZE)
print('EMBEDDING_DIMENS: ', EMBEDDING_DIMENS)

GRU_UNITS = 100

In [ ]:
import numpy as np
import torch
from collections import Counter
import re
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import RMSprop,Adam

class answerMask(nn.Module):
    def __init__(self,seq_length=200):
        super().__init__()
        self.seq_length = 200    
        self.n_units = 128
        embedding_size = self.n_units #n_units에 맞추기
        total_words = 9984 # gloVe dim에 맞추기
        self.num_layers = 1 #적층 레이어를 위해
        self.batch_size =1

        self.embed = nn.Embedding(
            num_embeddings=total_words, embedding_dim= embedding_size
        )
        self.answer_mask_gru = nn.GRU(
            input_size=self.n_units,
            hidden_size=self.n_units,
            batch_first=True,
            dropout=0.2,            
        )
        self.answer_mask = nn.Sequential(
            nn.Linear(self.n_units,2), 
            nn.Softmax(), 
        )

        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = Adam(self.parameters(),lr=0.001)
    def forward(self,x,prev_state):  
        # [126, 200]
        print("x.shape",x.shape)
        emb_init = self.embed(x) 
        # [126, 200, 128] #prev_state [1, 126, 128]을 원함
        print("emb_init.shape",emb_init.shape)
        self.output,state =self.answer_mask_gru(emb_init,prev_state)
        # [126, 200, 128] [1, 126, 128]
        print("self.output.shape",self.output.shape)
        print("state.shape",state.shape)
        mask = self.answer_mask(self.output)
        # [126, 200, 2]
        print("mask.shape",mask.shape)
        return mask 

    def init_state(self):
        return torch.zeros(self.num_layers, self.batch_size, self.n_units)
    
    def build_loss(self,document,answer_label):
        self.batch_size = document.shape[0]
        state_h = self.init_state()
        self.optimizer.zero_grad()

        y_pred = self.forward(document, state_h)
        y_pred = y_pred.view(-1,2)
        loss = self.loss_fn(y_pred, answer_label)

        # loss.backward()
        # self.optimizer.step()

        return loss
        
        


In [ ]:
class QnAEncoderDecoderModel(nn.Module):
    def __init__(self,seq_length=20):
        super().__init__()
        
        # n_units = 256
        # embedding_size = 100
        self.seq_length =200
        self.n_units = 128
        self.num_layers = 1 #적층 레이어를 위해
        self.total_words = 9984 # gloVe dim에 맞추기
        
        self.encoder_gru = nn.GRU(
            input_size=self.n_units,
            hidden_size=self.n_units,
            batch_first=True,
            dropout=0.2,            
        )

        self.decoder_gru = nn.GRU(
            input_size=self.n_units,
            hidden_size=self.n_units,
            batch_first=True,
            dropout=0.2,            
        )

        self.fc = nn.Sequential(
            nn.Linear(self.n_units,self.total_words),
            nn.Softmax(),
        )      
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = Adam(self.parameters(),lr=0.001)
        self.dec_state =None #나중에 질문 생성을 위해

    def forward(self,encoder_inputs,question,prev_state):  
        # encoder_inputs [126, 25, 128]  #h_0 [1, 126, 128]을 원함. [1, 25, 128]
        print("encoder_inputs.shape",encoder_inputs.shape)
        print("question.shape",question.shape)
        enc_output,enc_state = self.encoder_gru(encoder_inputs,prev_state)
        # [126, 25, 128], [1, 126, 128]
        print("enc_output.shape",enc_output.shape)
        print("enc_state.shape",enc_state.shape)
        # question [126, 13]
        emb_question = self.embed(question).detach()
        # [126, 13, 128]
        print("emb_question.shape",emb_question.shape)
        dec_output,self.dec_state = self.decoder_gru( emb_question ,enc_state)
        # [126, 13, 128], [1, 126, 128]
        print("dec_output.shape",dec_output.shape)
        print("dec_state.shape",self.dec_state.shape)

        logits = self.fc(dec_output) 
        print("logits.shape",logits.shape)
        #[126, 13, 9984]
        return logits
    
    def init_state(self):
        return torch.zeros(self.num_layers, self.batch_size, self.n_units)

    def answerMaskDotAnswerOutputs(self,answer_mask_label,answer_outputs):
        # [119, 51, 200] x [119, 200, 512]
        encoder_inputs = torch.bmm(answer_mask_label,answer_outputs)
        # [119, 51, 512]
        return encoder_inputs

    def predictY(self,answer_mask_model,question_input_token,answer_mask_label):
        self.embed = answer_mask_model.embed
        
        encoder_inputs = self.answerMaskDotAnswerOutputs(answer_mask_label,answer_mask_model.output)
        
        self.batch_size=encoder_inputs.shape[0]
        state_h = self.init_state()
        self.optimizer.zero_grad()
        
        y_pred = self.forward(encoder_inputs,question_input_token, state_h)
        return y_pred

    def build_loss(self,answer_mask_model,question_input_token,question_output_token,answer_mask_label):
        y_pred = self.predictY(answer_mask_model,question_input_token,answer_mask_label)
        y_pred = y_pred.view(-1,self.total_words)
        loss = self.loss_fn(y_pred, question_output_token)

        # loss.backward()
        # self.optimizer.step()

        return loss

In [ ]:
answer_mask_model = answerMask()
qna_model = QnAEncoderDecoderModel()

In [ ]:
import numpy as np

training_loss_hist = []
test_loss_hist = []

#EPOCHS = 2000
EPOCHS = 1
start_epoch = 1
for epoch in range(start_epoch, start_epoch + EPOCHS + 1):
    print("에포크 {0}".format(epoch))
    
    for i, batch in enumerate(training_data()):
        
        val_batch = next(test_data_gen, None)
        
        if val_batch is None:
            test_data_gen = test_data()
            val_batch = next(test_data_gen, None)
        print('hi')
        answer_training_loss = answer_mask_model.build_loss(
            torch.LongTensor( batch['document_tokens']),
            torch.LongTensor(batch['answer_labels'].flatten()),
        )
        print('===hi===')
        decoder_training_loss = qna_model.build_loss(
            answer_mask_model,
            torch.LongTensor(batch['question_input_tokens']),
            torch.LongTensor(batch['question_output_tokens'].flatten()),
            torch.FloatTensor( batch['answer_masks']),
        )
        
    
        
        total_loss = answer_training_loss+ decoder_training_loss
        training_loss_hist.append(total_loss.item())
        total_loss.backward()
        answer_mask_model.optimizer.step()
        qna_model.optimizer.step()


        print(f"{i}: 훈련 손실: {total_loss.item():.4f}")    
        if i==1:
            break
#    if epoch % 100 == 0:
#        print(f"{i}: 훈련 손실: {total_loss:.4f}")    
    break

## 테스트  
gan gdl 참고 후 수정

In [ ]:
test_data_gen = test_data()
batch = next(test_data_gen)
batch = collapse_documents(batch)
idx=0
print(batch['document_words'][idx][37:50])
for i in range(len(batch['document_words'][idx])):
    print(i, batch['document_words'][idx][i])

In [ ]:
import matplotlib.pyplot as plt

# 대답 위치 예측

plt.figure(figsize=(15,5))
idx = 0

answer_mask_model.batch_size = torch.LongTensor( batch['document_tokens']).shape[0]
answer_preds = answer_mask_model(torch.LongTensor( batch['document_tokens']),answer_mask_model.init_state())

print('예측한 대답의 확률')
ax = plt.gca()
ax.xaxis.grid(True)
plt.plot(answer_preds[idx, :, 1].detach().numpy())
plt.show()

for i in range(len(batch['document_words'][idx])):
    print(i, batch['document_words'][idx][i], np.round(answer_preds[idx][i][1].detach().numpy(),2))

In [ ]:
# 대답 위치 선정

start_answer = 37
end_answer = 39

print(batch['document_words'][idx][start_answer:(1+end_answer)])

In [ ]:
from utils.write import START_TOKEN, END_TOKEN, look_up_token

answer_mask_model.batch_size = torch.LongTensor( batch['document_tokens']).shape[0]
answer_preds = answer_mask_model(torch.LongTensor( batch['document_tokens']),answer_mask_model.init_state())

answers = [[0] * len(answer_preds[idx])]
for i in range(start_answer, end_answer + 1):
    answers[0][i] = 1

answer_batch = expand_answers(batch, answers)
with torch.no_grad():
    qna_model.predictY(
        answer_mask_model,
        torch.LongTensor( answer_batch['document_tokens'][[idx]]), 
        torch.FloatTensor(answer_batch['answer_masks'][[idx]]))
    next_decoder_init_state = qna_model.dec_state
    print("next_decoder_init_state.shape",next_decoder_init_state.shape)

    word_tokens = [START_TOKEN]
    questions = [look_up_token(START_TOKEN)]

    ended = False
    counter = 0

    while not ended:
        
        counter += 1
        #print("torch.LongTensor(word_tokens)",torch.LongTensor(word_tokens).unsqueeze(1).shape)
        emb_output=qna_model.embed(torch.LongTensor(word_tokens).unsqueeze(1))
        #print("emb_output",emb_output.shape)
        decoder_output, next_decoder_init_state = qna_model.decoder_gru( 
            emb_output,
            next_decoder_init_state)
        
        word_preds = qna_model.fc(decoder_output)
        #print("word_preds.shape",word_preds.shape)
        word_tokens = torch.argmax(word_preds, 2)[0]
        print("look_up_token(word_tokens[0].detach().numpy())",look_up_token(word_tokens[0].detach().numpy()))
        questions.append(look_up_token(word_tokens[0].detach().numpy()))

        if word_tokens[0] == END_TOKEN or counter > 20 :
            ended = True

    questions = ' '.join(questions)